Plusieurs types d'indicateurs vont être construits :

- des indicateurs de performances récentes
- des indicateurs de performances passées, sur la base des données d'historisation récupérées
- des indicateurs sur les confrontations directes entre les deux clubs
- des facteurs externes pouvant influencer le résultat d'un match (TODO)

In [3]:
import os
import sys
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_path)

import pandas as pd

from src.config import load_config
from src.preprocessing import Preprocessing

# config.yaml importation
config_file = 'config.yaml'
config_path = os.path.join(root_path, config_file)
config = load_config(config_path)

In [7]:
cleaned_data_path = os.path.join(root_path, config['cleaning_dir'])
train_df = pd.read_csv(cleaned_data_path+'/cleaned_df_train.csv')

In [15]:
test = Preprocessing(train_df, config)
test.creation_betting_odd_variable()

,date,home,away,nb_goals_home,nb_goals_away,final_result,home_odd,away_odd,draw_odd
0,2012-08-10,Montpellier,Toulouse,1.0,1.0,draw,1.833333,4.686667,3.335000
1,2012-08-11,ETG,Bordeaux,2.0,3.0,away,2.821667,2.575000,3.166667
2,2012-08-11,Nancy,Brest,1.0,0.0,home,2.118333,3.756667,3.126667
3,2012-08-11,Nice,Ajaccio,0.0,1.0,away,1.960000,4.250000,3.228333
4,2012-08-11,PSG,Lorient,2.0,2.0,draw,1.336667,10.195000,4.646667
...,...,...,...,...,...,...,...,...,...
4380,2024-05-19,Lyon,Strasbourg,2.0,1.0,home,1.595821,6.523753,4.741060
4381,2024-05-19,Metz,PSG,0.0,2.0,away,4.647487,2.002086,4.231060
4382,2024-05-19,Monaco,Nantes,4.0,0.0,home,1.572487,6.502086,4.959393
4383,2024-05-19,Reims,Rennes,2.0,1.0,home,2.744154,2.723753,3.486060


In [15]:
1 != 2 != 3

True